In [1]:
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import pandas as pd 

"""this file is output by entity_annotation_for_RE.ipynb """
file = r'/data2/zhanghc/RE/low-resource/src/data/entity_annotation.csv'
df_r = pd.read_csv(file)

This is the schema used by relation labeling
They are tag combination
***********************************
Component: hasAttribute
Unit_Data_Storage
Dimensions
Unit_Weights
Unit_Speed
Unit_WriteSpeed
Unit_Resolution
Unit_electic_current
Time_Span
Size

Product: hasComponent
Component

Product: hasBrand
Brand

Product:hasFeature:
Feature

Product:hasSoftware:
Software
************************************

In [2]:
from allennlp.data.tokenizers import Tokenizer, WordTokenizer
from allennlp.data.tokenizers.word_splitter import JustSpacesWordSplitter
import re

_tokenizer =  WordTokenizer(word_splitter=JustSpacesWordSplitter())

POSITIVE = 1
NEGATIVE = 0

# build up the schema to check from
head_dict = {'COMPONENTS': {i for i in range(9)}, 'PRODUCT':{i for i in range(9,13)} }
tail_dict = {'UNIT_DATA_STORAGE': {0}, 'DIMENSIONS': {1}, 'UNIT_WEIGHTS': {2},
             'UNIT_SPEED': {3}, 'UNIT_WRITESPEED': {4}, 'UNIT_RESOLUTION': {5},
             'UNIT_ELECTIC_CURRENT':{ 6 }, 'TIME_SPAN': {7 }, 'SIZE':{ 8 },
            'COMPONENTS': {9 }, 'BRAND': {10 }, 'FEATURE': {11}, 'SOFTWARE': {12}}
relation_tag=["hasAttribute"]*9+["hasComponent","hasBrand","hasFeature","hasSoftware"]
             
def findIndex(text_array, token):
    if token in text_array:
        return text_array.index(token)
    else:
        for i, t in enumerate(text_array):
            if token in t:
                return i
        token_new = re.sub("\W$|^\W", "", token)
        if token_new in text_array:
            return text_array.index(token_new)
        else:
            return 0

def find_start_end(text_array, token1, token2):
    start = findIndex(text_array, token1)
    if token1 == token2:
        end = start
    else:
        end = findIndex(text_array, token2)
        if start >= end: end = start
    return start, end

            
    


In [3]:
from snorkel.labeling import labeling_function
import json
import random
from tqdm import tqdm

output_result = []
n_positive=0
total_n = 0
for row in tqdm(df_r.itertuples()):
    docid = row[2]
    ents = row[3]
    sentence = row[4]
    
    jents = json.loads(ents.replace("'", "\"" ))
    tags = [ ent["label"] for ent in jents]
    texts=[ ent["text"] for ent in jents]
    
    ent_num = len(jents)
  
    if ent_num < 2 : continue
    
    if len(str(sentence).split(" "))<5:continue
        
    text_new = sentence.split(" ")
     # to find out each entity pairs
    for i in range(ent_num):
        head = jents[i]
        head_text = sentence[head['start_char']:head['end_char']]
        head_token = head_text.split(" ")  # self._tokenizer.tokenize(head_text)
        head_start, head_end = find_start_end(text_new, head_token[0], head_token[-1])
        
        for j in range(ent_num):
            if i == j:continue
       
            tail = jents[j]
            
            tail_text = sentence[tail['start_char']:tail['end_char']]
            
            if str(head_text.lower()) == str(tail_text.lower()):continue
        
            tail_token = tail_text.split(" ")  
            tail_start, tail_end = find_start_end(text_new, tail_token[0], tail_token[-1])
            
            label_name = "None"
            if head['label'] in head_dict and tail['label'] in tail_dict:
                relation_ = head_dict[head['label']].intersection(tail_dict[tail['label']])
                label_name = relation_tag[relation_.pop()] if len(relation_)>0 else"None"
            
            if label_name == "hasAttribute":
                #Re-judge the attribute relationship of the component whether correctly labeled
                tag_span = tags[i+1:j] if i<j else tags[j+1:i]
                if "COMPONENTS" in tag_span:
                    re_label = "None"
         
            line_json={"id": docid, "sent": sentence, "label": label_name, "entities": [[head_start,head_end],[tail_start,tail_end]],
                       "head_text":head_text,
                  "tail_text":tail_text}
            
            """each line only contains one relation. the same sentence repeat in different pair of entitis."""
            
            output_result.append(line_json)  
            
            if label_name != "None":n_positive+=1
            total_n+=1

print("positive:"+str(n_positive))
print("porprotion:"+str(float(n_positive/total_n)) )           
random.shuffle(output_result)

"""we split train/dev dataset and write down"""

path_out_train="/data2/zhanghc/RE/low-resource/src/data/distantly_labeled/relation_annotation_train.csv"
path_out_dev="/data2/zhanghc/RE/low-resource/src/data/distantly_labeled/relation_annotation_dev.csv"
proportion_len = int(len(output_result)* 0.7)
with open(path_out_train, 'w') as out_f:
    for inst in output_result[:proportion_len]:
        out_f.write(json.dumps(inst) + '\n')
with open(path_out_dev, 'w') as out_f:
    for inst in output_result[proportion_len:]:
        out_f.write(json.dumps(inst) + '\n')
          
print("total:"+str(len(output_result)))
        


738456it [00:30, 23876.27it/s]


positive:349408
porprotion:0.2089113540183125
total:1672518
